### Log into ArcGIS Online

In [1]:
from arcgis.gis import GIS

import warnings
from urllib3.exceptions import InsecureRequestWarning
warnings.simplefilter("ignore", InsecureRequestWarning)

gis = GIS("home")

In [2]:
cm_id = "22036bc8cafb4c50b6276c73f8f81c8e"
cm_item = gis.content.get(cm_id)

monitoring_table = cm_item.tables[0]
dependencies_table = cm_item.tables[1]

In [3]:
def get_sharing(sharing):
    groups = len(sharing.shared_with["groups"])
    level = sharing.shared_with["level"].value
    if groups:
        if level == "PRIVATE":
            return "Groups only"
        elif level == "ORGANIZATION":
            return "Groups & Organization"
        elif level == "EVERYONE":
            return "Groups & Everyone"
    else:
        if level == "PRIVATE":
            return "Private"
        elif level == "ORGANIZATION":
            return "Organization"
        elif level == "EVERYONE":
            return "Everyone"

def get_status(status):
    if not status:
        return "None"
    else: return status.lstrip("org_").title()

def get_item_url(item):
    if item.type in ["Vector Tile Service", "Feature Service", "Map Service", "Image Service"]:
        return f'<a target="_blank" href="{item.url}">View Service</a>'
    elif item.type == "Web Map":
        return f'<a target="_blank" href="https://www.maps.arcgis.com/apps/mapviewer/index.html?webmap={item.id}">View Map</a>'
    elif item.type in ["Hub Site Application", "StoryMap", "Web Experience", "Web Experience Template", "Dashboard", "Hub Page"]:
        return f'<a target="_blank" href="{item.url}">View App</a>'
    else: return "Not applicable"

def get_item_page_url(item_id):
    return f'<a target="_blank" href="https://www.maps.arcgis.com/home/item.html?id={item_id}">View Item Page</a>'

In [4]:
all_items = gis.content.search(query="*", max_items=10000)

items_list = []
dependencies_list = []

for item in all_items:

    items_dict = {}
    update_items_dict = {}
    
    items_dict["item_id"] = item.id
    items_dict["item_title"] = item.title
    items_dict["item_owner"] = item.owner
    items_dict["item_type"] = item.type
    items_dict["item_page_url"] = get_item_page_url(item.id)
    items_dict["item_url"] = get_item_url(item)
    items_dict["item_sharing"] = get_sharing(item.sharing)
    items_dict["item_status"] = get_status(item.content_status)
    items_dict["date_created"] = item.created
    items_dict["date_modified"] = item.modified

    # BEGIN DEPENDENCIES
    dependencies = item.get_dependencies(deep=True)

    if not dependencies:
        items_dict["has_dependencies"] = "No"
        items_dict["dependencies_count"] = 0

    else:
        items_dict["has_dependencies"] = "Yes"
        items_dict["dependencies_count"] = len(dependencies)

        for dependency in dependencies:

            dependencies_dict = {}
            update_dependencies_dict = {}

            dependencies_dict["origin_id"] = item.id
            dependencies_dict["origin_title"] = item.title
            dependencies_dict["dependent_id"] = dependency.id
            dependencies_dict["dependent_title"] = dependency.title
            dependencies_dict["dependent_owner"] = item.owner
            dependencies_dict["dependent_type"] = dependency.type
            dependencies_dict["dependent_item_page_url"] = get_item_page_url(dependency.id)
            dependencies_dict["dependent_item_url"] = get_item_url(dependency)

            dependencies_dict["dependent_sharing"] = get_sharing(item.sharing)
            dependencies_dict["dependent_status"] = get_status(item.content_status)
            dependencies_dict["dependent_date_created"] = item.created
            dependencies_dict["dependent_date_modified"] = item.modified

            update_dependencies_dict["attributes"] = dependencies_dict
            dependencies_list.append(update_dependencies_dict)

    update_items_dict["attributes"] = items_dict
    items_list.append(update_items_dict)

print(f"Number of items to add to items table: {len(items_list)}")
print(f"Number of items to add to dependencies table: {len(dependencies_list)}")

Number of items to add to items table: 225
Number of items to add to dependencies table: 403


### Add dependencies to the dependencies table

In [5]:
# This script will effectively overwrite the output of the last run, 
# so before writing any new records we simply truncate tables
# NOTE: can't use truncate like for users table because of the relationship class!
# NOTE: Do we even need the relationship class? We're just shoving everything in a Dashboard/ExB anyway...?
monitoring_table.delete_features(where="1=1")
items_update = monitoring_table.edit_features(adds=items_list)

# Add items and dependencies to tables
dependencies_table.delete_features(where="1=1")
dependencies_update = dependencies_table.edit_features(adds=dependencies_list)